# Multilayer perceptron NN with Pytorch

In [80]:
import math
import random
import torch

Define values with torch

In [93]:
x1 = torch.Tensor([1.0]).double()
x2 = torch.Tensor([0.0]).double()
w1 = torch.Tensor([-1.0]).double()
w2 = torch.Tensor([1.0]).double()
b = torch.Tensor([10]).double()

x1.requires_grad = True
x2.requires_grad = True
w1.requires_grad = True
w2.requires_grad = True
b.requires_grad = True

# neuron
n = x1*w1 + x2*w2 + b
# activation
o = torch.tanh(n)

Value class for all basic operations

In [110]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __rmul__(*self, other):
    return self * other

  def __truediv__(self, other):
    return self * other**-1

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def __radd__(self, other):
    return self + other

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "support int/float power atm"
    out = Value(self.data**other, (self, ), f'**{other}')

    def _backward():
      self.grad += (other * self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def backward(self):

    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()



Class Neuron for an object x*w + b

In [117]:

class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def params(self):
    return self.w + [self.b]


Class Layer - layer in Neural Network

In [118]:
class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def params(self):
    return [p for neuron in self.neurons for p in neuron.params()]


Main class for MLP NN

In [119]:
class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def params(self):
    return [p for layer in self.layers for p in layer.params()]


In [120]:
# initialize NN

x = [5.0, 8.0, -1.0]
n = MLP(3, [3, 4, 1])
n(x)

Value(data=-0.5771440677905261)

In [121]:
# gradient descent

# input values
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

# target prediction
ys = [1.0, -1.0, -1.0, 1.0]

for k in range(20):

  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - yg)**2 for yg, yout in zip(ys, ypred))

  # backward pass
  for p in n.params():
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.params():
    p.data += -0.05 * p.grad

  print(k, loss.data)


0 4.586615659430144
1 1.6616804193656831
2 1.0144258156051902
3 0.6586138198536479
4 0.47634881884248664
5 0.37030342112367143
6 0.3006496172583954
7 0.25187945046415133
8 0.2160684101954755
9 0.18877554227578536
10 0.1673481916033126
11 0.1501157224785843
12 0.13597863993597026
13 0.12418616609143562
14 0.11420940287443909
15 0.10566569506094342
16 0.09827175112588563
17 0.0918135978903587
18 0.08612675159869174
19 0.08108278952564478
